<a href="https://colab.research.google.com/github/adagio7/induction-heads/blob/main/induction_heads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
%pip install transformer-lens
%pip install circuitsvis

In [3]:
import torch
import circuitsvis as cv
import transformer_lens.utils as utils
from transformer_lens import (
    HookedTransformer
)

# We are only really interested in model inference, not training
torch.set_grad_enabled(False)
device = utils.get_device()

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

In [5]:
prompt = "Tom went to the store. Tom"
tokens = model.to_tokens(prompt)

logits, cache = model.run_with_cache(tokens, remove_batch_dim=True)
print(logits.shape)

torch.Size([1, 8, 50257])


In [9]:
attention_pattern = cache["pattern", 6, "attn"]
gpt2_str_tokens = model.to_str_tokens(prompt)

print(f'{attention_pattern.shape=}')

# Note that `attention_patterns` is deprecated, and `attention_heads` should be used instead
# But for space, we used the old version as its more compact
cv.attention.attention_patterns(tokens=gpt2_str_tokens, attention=attention_pattern)

attention_pattern.shape=torch.Size([12, 8, 8])
